In [18]:
import csv
import numpy as np
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [19]:
def load_csv_to_trajectory(filepath):
    """Lädt die CSV und stellt Trajektorie-Liste wieder her"""
    trajectory = []
    with open(filepath, mode='r', newline='') as file:
        reader = csv.reader(file)
        next(reader)  # Überspringe Kopfzeile

        for row in reader:
            numbers = list(map(float, row))
            trans = numbers[:3]
            rot = numbers[3:]
            trajectory.append((trans, rot))
    return trajectory

In [20]:
traj_1_path = '/home/mo/Thesis/Evaluation/Trajektorientests/CSV/trajectory_1.csv'
traj_2_path = '/home/mo/Thesis/Evaluation/Trajektorientests/CSV/trajectory_2.csv'
traj_3_path = '/home/mo/Thesis/Evaluation/Trajektorientests/CSV/trajectory_3.csv'
traj_4_path = '/home/mo/Thesis/Evaluation/Trajektorientests/CSV/trajectory_4.csv'

In [21]:
trajectory_1 = load_csv_to_trajectory(traj_1_path)
trajectory_2 = load_csv_to_trajectory(traj_2_path)
trajectory_3 = load_csv_to_trajectory(traj_3_path)
trajectory_4 = load_csv_to_trajectory(traj_4_path)

In [86]:
def compute_rectangle_corners(trajectory, width=3, height=4):
    """
    Berechnet die Rechteck-Eckpunkte (4 Stück) für jeden Trajektorienpunkt.

    :param trajectory: Liste von ([x, y, z], [roll, pitch, yaw]) Einträgen
    :param width: Breite des Rechtecks (in X-Richtung)
    :param height: Höhe des Rechtecks (in Y-Richtung)
    :return: Liste von Rechtecken, jedes als Liste von 4 Eckpunkten (jeweils [x, y, z])
    """
    rectangles = []

    # Rechteck im lokalen TCP-Koordinatensystem (XY-Ebene, Zentrum im Ursprung)
    half_w = width / 2
    half_h = height / 2
    local_corners = np.array([
        [-half_w, 0, -half_h],
        [ half_w, 0, -half_h],
        [ half_w, 0, half_h],
        [-half_w, 0, half_h]
    ])

    for trans, rpy in trajectory:
        # RPY in Radiant, falls nicht schon so
        rot = R.from_euler('xyz', rpy, degrees=True)
        rotated_corners = rot.apply(local_corners)
        translated_corners = rotated_corners + np.array(trans)
        rectangles.append(translated_corners.tolist())
    return rectangles

In [ ]:
rectangles_1 = compute_rectangle_corners(trajectory_1)
rectangles_2 = compute_rectangle_corners(trajectory_2)
rectangles_3 = compute_rectangle_corners(trajectory_3)
rectangles_4 = compute_rectangle_corners(trajectory_4)

In [116]:
def visualize_rectangles(rectangles, figsize=(6, 6), connect_points=None):
    """
    Visualisiert Rechtecke + Trajektorie im 3D-Raum mit Legende und Bezeichnungen der Trajektorienpunkte.

    :param rectangles: Liste von Rechtecken (jeweils 4 Punkte [x, y, z])
    :param figsize: Größe der Plotfläche (Breite, Höhe)
    :param connect_points: Optionales Tupel mit zwei Punkten [x, y, z], die verbunden werden sollen
    """
    %matplotlib qt
    
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d.art3d import Poly3DCollection
    import numpy as np
    import matplotlib.cm as cm

    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111, projection='3d')

    centers = []

    # Farbbereiche erstellen (für unterschiedliche Rechtecke)
    cmap = cm.get_cmap('tab20')  # Eine colormap, die mehrere Farben bereitstellt

    # Liste für die Handles und Labels der Legende
    legend_handles = []

    # Labels für die Trajektorienpunkte
    labels = ['Pre-Position', 'Spitze', 'Senke']

    for i, rect in enumerate(rectangles):
        color = cmap(i / len(rectangles))  # Farbwahl basierend auf dem Index des Rechtecks

        poly = Poly3DCollection([rect], facecolors=color, alpha=0.6, edgecolors=color)
        ax.add_collection3d(poly)

        # Einen Dummy-Punkt für die Legende hinzufügen
        if i == 0:
            label = 'Pre-Position'
        elif i == 1:
            label = 'Spitze'
        elif i == len(rectangles) - 1:
            label = 'Senke'
        else:
            label = str(i + 1)  # Für die anderen Rechtecke (die nicht Pre-Position, Spitze oder Senke sind)

        legend_handles.append(plt.Line2D([0], [0], marker='o', color='w', label=label,
                                         markersize=10, markerfacecolor=color))

        center = np.mean(np.array(rect), axis=0)
        centers.append(center)
        ax.scatter(*center, color='red', s=30)

    # Trajektorienlinie (Mittelpunkte)
    centers = np.array(centers)
    ax.plot(centers[:, 0], centers[:, 1], centers[:, 2], color='black', linewidth=2)

    # Bezeichne die Trajektorienpunkte
    if len(centers) > 0:
        ax.text(centers[0][0], centers[0][1], centers[0][2], 'Pre-Position', color='black', fontsize=10)
    if len(centers) > 1:
        ax.text(centers[1][0], centers[1][1], centers[1][2], 'Spitze', color='black', fontsize=10)
    if len(centers) > 2:
        ax.text(centers[-1][0], centers[-1][1], centers[-1][2], 'Senke', color='black', fontsize=10)

    # Optional: Verbindung zweier beliebiger Punkte in grau gestrichelt
    if connect_points is not None and len(connect_points) == 2:
        p1, p2 = np.array(connect_points[0]), np.array(connect_points[1])
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]],
                color='gray', linestyle='dashed', linewidth=2)
        ax.scatter(p1[0], p1[1], p1[2], 'o', color='magenta', linewidth=6)
        ax.scatter(p2[0], p2[1], p2[2], 'o', color='magenta', linewidth=6)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Rechtecke im Raum (TCP Orientierung)')

    # Automatische gleichmäßige Skalierung
    all_points = np.vstack(rectangles)
    mid = np.mean(all_points, axis=0)
    max_range = np.ptp(all_points, axis=0).max() / 2

    for i, axis in enumerate([ax.set_xlim, ax.set_ylim, ax.set_zlim]):
        axis(mid[i] - max_range, mid[i] + max_range)

    # Gleichmäßige Skalierung der Achsen
    ax.set_box_aspect([1, 1, 1])  # Setzt gleiche Skalierung für alle Achsen

    # Legende mit den Bezeichnern Pre-Position, Spitze und Senke
    ax.legend(handles=legend_handles, title="Rechtecke", loc='upper left')

    plt.tight_layout()
    plt.show()

In [118]:
visualize_rectangles(rectangles_1, connect_points = (trajectory_1[1][0], trajectory_1[-1][0]))
visualize_rectangles(rectangles_2, connect_points = (trajectory_1[1][0], trajectory_1[-1][0]))
visualize_rectangles(rectangles_3, connect_points = (trajectory_1[1][0], trajectory_1[-1][0]))
visualize_rectangles(rectangles_4, connect_points = (trajectory_1[1][0], trajectory_1[-1][0]))

/tmp/ipykernel_118458/3002180928.py:22: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('tab20')  # Eine colormap, die mehrere Farben bereitstellt
